# Pipeline using application/ld+json works for
- nos.nl (getting search results is possible and loopable)
- cnn (getting search results is possible and loopable)
- fox news (verified freely accessible)
- cbs news (but getting search results is hard)
- nbc news

# Pipeline using application/ld+json does not work for
- abc news, which should be accessed via the "body" element of another script (which unfortunately has no name)
- 

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import time

In [11]:
#Load page
driver = webdriver.Chrome()
driver.get("https://www.foxnews.com/search-results/search?q=AI")
time.sleep(5)

In [ ]:
#Close the banner in front of load-more
element = driver.find_element(By.CSS_SELECTOR, ".close")
element.click()

In [8]:
#Code to click the load-more button on Fox News search results page
for i in range(30):
    element = driver.find_element(By.CSS_SELECTOR, ".button.load-more")
    element.click()
    time.sleep(1)

In [9]:
# Gets all links on a page using selenium
elements = driver.find_elements(By.TAG_NAME, "a")
for element in elements:
    print(element.get_dom_attribute("href"))

https://www.foxnews.com/
https://www.foxbusiness.com/
https://nation.foxnews.com/
https://radio.foxnews.com/
https://www.foxweather.com/
https://www.outkick.com/
https://www.foxnews.com/books
https://www.foxnews.com/
/us
/politics
/world
/opinion
/media
/entertainment
/sports
/lifestyle
/video
/category/tech/artificial-intelligence
#
#
#
https://www.foxnews.com/video/5614615980001
#
https://www.foxnews.com/politics/ny-v-trump-jury-verdict-former-presidents-historic-criminal-trial
https://www.foxnews.com/politics/trump-vows-fight-end-promising-this-long-from-over-after-guilty-verdict
https://www.foxnews.com/politics/trump-ny-sentencing-days-before-republican-convention
https://www.foxnews.com/live-news/may-30-trump-trial-jury-deliberations-day-2
//www.foxnews.com/us
//www.foxnews.com/category/us/crime
//www.foxnews.com/category/us/military
//www.foxnews.com/category/us/education
//www.foxnews.com/category/us/terror
//www.foxnews.com/category/us/immigration
//www.foxnews.com/category/us/

In [ ]:
from bs4 import BeautifulSoup
from html import parser
import re

In [ ]:
#Gets all search hit pages from searching for AI on NOS.nl
d = {}
for i in range(30)[1:]:
    with open(f"pages/nos-ai-{i}.html") as fp:
        soup = BeautifulSoup(fp, 'html.parser')
        d[i] = soup

In [ ]:
#Gets all relevant_links from the search results for AI on NOS.nl
relevant_links = []

for i in d.keys():
    record = False
    zoeken_counter = 0
    for link in d[i].find_all('a'):
        if link.get('href')[:8] == "/zoeken?":
            record = False
        if record and re.search(r"artikel",link.get("href")):
            relevant_links.append("https://www.nos.nl" + link.get('href'))
        if link.get('href') == "/zoeken":
            zoeken_counter+=1
        if zoeken_counter == 2:
            record = True
            zoeken_counter = 0

In [ ]:
help(BeautifulSoup)

In [ ]:
#Gets the soup for all files in relevant_links[:10]
import os
soups = []
for link in relevant_links[:10]:
    os.system(f"wget -O link.html {link}")
    with open("link.html","r") as file:
        soups.append(BeautifulSoup(file, 'html.parser'))

In [ ]:
#Get the json-file containing file content and metadata for all soup in soups
#and turn into dictionary (this has been tested for NOS.nl and for one article on CNN.com; 'application/ld+json' seems to be a standard
#type for scripts describing the metadata and content of news site articles)
import json
contents = []
for soup in soups:
    for script in soup.find_all("script"):
        try:
            if script['type'] == 'application/ld+json':
                contents.append(json.loads(script.contents[0]))
        except:
            pass
date = contents[9]['datePublished']
headline = contents[9]['headline']
mainText = contents[9]['articleBody']
date, headline, mainText